In [1]:
import numpy as np 
import pandas as pd
import Bio
from Bio.pairwise2 import format_alignment
import copy

In [2]:
filename = 'ECas9_cleavage_rate_and_y0.txt'
Path = '../data_nucleaseq_Finkelsteinlab/targetE/'
data = pd.read_table(Path+filename, usecols=['target', 'cleavage_rate', 'cleavage_rate_5th_pctl', 'cleavage_rate_95th_pctl'])
data.rename(columns={'target':'Sequence'}, inplace=True)

In [4]:
data.head()

,Sequence,cleavage_rate,cleavage_rate_5th_pctl,cleavage_rate_95th_pctl
0,TTTAGAGCGTATTTCATGAGACGCTGG,2.657239e-06,6.222793e-24,0.000004
1,TTTAGACTGCGATAAAGATGAGACGCTGG,2.767674e-22,1.326366e-25,0.000005
2,TTTAGACGCATAAAGATGAGACGCCTTAA,1.199120e-06,1.717058e-27,0.000002
3,TTTAGACGCATAAAGATGAGACGCTAGGA,5.308185e-05,4.825889e-05,0.000069
4,TTTAGACGTCAGTAAAGATGAGACGCTGG,8.747661e-25,7.856313e-27,0.000001


In [84]:
def separate_PAM(S, Cas='Cas9'):
    
    if Cas == 'Cas9':
        PAM_len =3
        PAM = S[-PAM_len:]
        s = S[0:-PAM_len]
        s = s[::-1]
        s = s[0:-4]
        canonical = False
        if PAM[1:]=='GG':
            canonical = True
    if Cas == 'Cas12a':
        PAM_len = 4
        PAM = S[:PAM_len]
        s = S[PAM_len:]
        s = s[0:-3]
        canonical = False
        if (PAM[:3]=='TTT')&(PAM[3]!='T'):
            canonical = True
    return PAM, s, canonical

def find_length_diff(S, on_target, Cas='Cas9'):
    
    _, s, canonical = separate_PAM(S, Cas)
    _, t, _ = separate_PAM(on_target, Cas='Cas9')
    return (len(s)-len(t))

def find_ontarget(S, on_target, Cas='Cas9'):
    
    _, s, _ = separate_PAM(S, Cas)
    _, t, _ = separate_PAM(on_target, Cas='Cas9')
    return s==t

def Allign(S, on_target, Cas):
    
    _, s, _ = separate_PAM(S, Cas)
    _, t, _ = separate_PAM(on_target, Cas)
    A = Bio.pairwise2.align.globalxx(t, s)
    if len(s)==len(t):
        MM_num = np.sum(np.array(list(t))==np.array(list(s)))
        a = (t,s,MM_num,0,len(t))
        if a not in A:
            A.append(a)
    
    return A

def Clean_allignment(x):
    A = copy.deepcopy(x['Alignment_raw'])
    Length_diff = x['Length difference']

    if Length_diff == 0:
        function_to_filter = lambda a: not(('-' in a[0]) or ('-' in a[1]))
        A = filter(function_to_filter, A)
        return A
    
    if Length_diff > 0:
        function_to_filter = lambda a: not((a[0].count('-')!=np.abs(Length_diff)) or ('-' in a[1]))
        A = filter(function_to_filter, A)
        id_for_sort = lambda a: '|'.join(map(lambda x: str(x), list(np.arange(1,len(a[0])+1)[np.array(list(a[0])) != np.array(list(a[1]))])))
        if len(A)>1:        
            A.sort(key=id_for_sort, reverse=True)
            del A[1:]
        return(A)
    
    if Length_diff < 0:
        function_to_filter = lambda a: not(('-' in a[0]) or (a[1].count('-')!=np.abs(Length_diff)))
        A = filter(function_to_filter, A)
        id_for_sort = lambda a: '|'.join(map(lambda x: str(x), list(np.arange(1,len(a[0])+1)[np.array(list(a[0])) != np.array(list(a[1]))])))
        if len(A)>1:        
            A.sort(key=id_for_sort, reverse=True)
            del A[1:]
        return(A)

def Make_Mutation_ID(x,seq_colname,on_target):
    
    _, t, _ = separate_PAM(on_target, Cas='Cas9')
    _, s, _ = separate_PAM(x[seq_colname], Cas)
    
    if x['On Target']:
        return 'OT'
    
    if len(x['Alignment_selected']) == 0:
        return ''
    
    Length_diff = int(x['Length difference'])
    a = x['Alignment_selected'][0]

    if Length_diff > 0:
        ta = np.array(list(a[0]))
        sa = np.array(list(a[1]))
        All_positions = np.arange(1,len(sa)+1)
        Mut_positions_on_s = All_positions[ta!=sa]
        Mut_Seqs = sa[[ta!=sa]]
        Mut_types = np.array(['i']*len(Mut_Seqs))
        Mut_Seqs_on_t = ta[[ta!=sa]]
        Mut_types[Mut_Seqs_on_t!='-'] = 'r'
        offset = np.zeros(len(Mut_types), dtype=int)
        for n in range(len(offset)):
            offset[n] = np.sum(Mut_types[0:n]=='i')
        # For insertion it gives the position of a base on the target before which there is an insertion.
        Mut_positions = Mut_positions_on_s - offset 
        ID_list = []
        for Mut_type, Mut_pos, Mut_Seq in zip(Mut_types, Mut_positions, Mut_Seqs):
            ID_list.append(':'.join([Mut_type, str(Mut_pos), Mut_Seq]))
        ID_list = filter(lambda x: 'i:21' not in x[:-2], ID_list) # Insertion after position 20 does not count.
        ID = '|'.join(ID_list)
        if ((ID=='')&(s[:-Length_diff]==t)):
            ID = 'OT' 

    if Length_diff < 0:
        ta = np.array(list(a[0]))
        sa = np.array(list(a[1]))
        All_positions = np.arange(1,len(ta)+1)
        Mut_positions = All_positions[ta!=sa]
        Mut_Seqs = ta[[ta!=sa]]
        Mut_types = np.array(['d']*len(Mut_Seqs))
        Mut_Seqs_on_s = sa[[ta!=sa]]
        Mut_types[Mut_Seqs_on_s!='-'] = 'r'
        Mut_Seqs[Mut_Seqs_on_s!='-'] = Mut_Seqs_on_s[Mut_Seqs_on_s!='-']
        ID_list = []
        for Mut_type, Mut_pos, Mut_Seq in zip(Mut_types, Mut_positions, Mut_Seqs):
            ID_list.append(':'.join([Mut_type, str(Mut_pos), Mut_Seq]))
        ID = '|'.join(ID_list)

    if Length_diff == 0:
        ta = np.array(list(a[0]))
        sa = np.array(list(a[1]))
        All_positions = np.arange(1,len(ta)+1)
        Mut_positions = All_positions[ta!=sa]
        Mut_Seqs = sa[[ta!=sa]]
        Mut_types = np.array(['r']*len(Mut_Seqs))
        ID_list = []
        for Mut_type, Mut_pos, Mut_Seq in zip(Mut_types, Mut_positions, Mut_Seqs):
            ID_list.append(':'.join([Mut_type, str(Mut_pos), Mut_Seq]))
        ID = '|'.join(ID_list)
    
    return ID


In [5]:
## Arguments
Cas='Cas9'
on_target = 'TTTAGACGCATAAAGATGAGACGCTGG'
seq_colname = 'Sequence'
output_colnames = ['cleavage_rate', 'cleavage_rate_5th_pctl', 'cleavage_rate_95th_pctl']

In [93]:
# function body
new_data = data.copy()
new_data['On Target'] = new_data[seq_colname].apply(lambda S: find_ontarget(S, on_target, Cas))
new_data['PAM'] = new_data[seq_colname].apply(lambda S: separate_PAM(S, Cas)[0])
new_data['Canonical'] = new_data[seq_colname].apply(lambda S: separate_PAM(S, Cas)[2])
new_data['Length difference'] = new_data[seq_colname].apply(lambda S: find_length_diff(S, on_target, Cas))
new_data['Alignment_raw'] = new_data[seq_colname].apply(lambda S: Allign(S, on_target, Cas))
new_data['Alignment_selected'] = new_data.apply(Clean_allignment, axis=1)
new_data['Alignment'] = new_data['Alignment_selected'].apply(lambda A: format_alignment(*A[0]).split('S')[0] if len(A) > 0 else '')
new_data['Alignment (All)'] = new_data['Alignment_raw'].apply(lambda A: ('\n').join(map(lambda a: format_alignment(*a).split('S')[0], A)) if len(A)>0 else '')
new_data['Mutation ID'] = new_data.apply(lambda x: Make_Mutation_ID(x,seq_colname,on_target), axis=1)
new_data['Mutation Type'] = new_data['Mutation ID'].apply(lambda x: '|'.join(list(set(map(lambda y: y[0], x.split('|'))))) if ((x!='')&(x!='OT')) else '')
new_data['Mutation Count'] = new_data['Mutation ID'].apply(lambda x: len(x.split('|')) if ((x!='')&(x!='OT')) else np.NaN)
new_data.loc[new_data['Mutation ID']=='OT','On Target'] = True
new_data.loc[new_data['On Target'],'Mutation Count'] = 0
new_data.loc[new_data['On Target'],'Mutation Type'] = 'OT'


In [94]:
new_data

,Sequence,cleavage_rate,cleavage_rate_5th_pctl,cleavage_rate_95th_pctl,On Target,PAM,Canonical,Length difference,Alignment_raw,Alignment_selected,Alignment,Alignment (All),Mutation ID,Mutation Type,Mutation Count
0,TTTAGAGCGTATTTCATGAGACGCTGG,2.657239e-06,6.222793e-24,3.658991e-06,False,TGG,True,0,"[(CGCAGAGTAGAAA-T--AC-GC-AG, CGCAGAGT----ACTTT...","[(CGCAGAGTAGAAATACGCAG, CGCAGAGTACTTTATGCGAG, ...",CGCAGAGTAGAAATACGCAG\n|||||||||.........||\nCG...,CGCAGAGTAGAAA-T--AC-GC-AG\n|||||||| | | | ...,r:10:C|r:11:T|r:12:T|r:13:T|r:14:A|r:15:T|r:16...,r,9.0
1,TTTAGACTGCGATAAAGATGAGACGCTGG,2.767674e-22,1.326366e-25,4.831201e-06,False,TGG,True,2,"[(CGCAGAGTAGAAATA-CG-CAG, CGCAGAGTAGAAATAGCGTC...","[(CGCAGAGTAGAAATA-CG-CAG, CGCAGAGTAGAAATAGCGTC...",CGCAGAGTAGAAATA-CG-CAG\n||||||||||||||| || |||...,CGCAGAGTAGAAATA-CG-CAG\n||||||||||||||| || |||...,i:16:G|i:18:T,i,2.0
2,TTTAGACGCATAAAGATGAGACGCCTTAA,1.199120e-06,1.717058e-27,2.334385e-06,False,TAA,False,2,"[(-C-GCAGAGTAGAAATACGCAG, TCCGCAGAGTAGAAATACGC...","[(-C-GCAGAGTAGAAATACGCAG, TCCGCAGAGTAGAAATACGC...",-C-GCAGAGTAGAAATACGCAG\n | |||||||||||||||||||...,-C-GCAGAGTAGAAATACGCAG\n | |||||||||||||||||||...,i:1:T|i:2:C,i,2.0
3,TTTAGACGCATAAAGATGAGACGCTAGGA,5.308185e-05,4.825889e-05,6.856400e-05,False,GGA,False,2,"[(--CGCAGAGTAGAAATACGCAG, ATCGCAGAGTAGAAATACGC...","[(--CGCAGAGTAGAAATACGCAG, ATCGCAGAGTAGAAATACGC...",--CGCAGAGTAGAAATACGCAG\n ||||||||||||||||||||...,--CGCAGAGTAGAAATACGCAG\n ||||||||||||||||||||...,i:1:A|i:1:T,i,2.0
4,TTTAGACGTCAGTAAAGATGAGACGCTGG,8.747661e-25,7.856313e-27,1.005484e-06,False,TGG,True,2,"[(CGCAGAGTAGAAAT-AC-GCAG, CGCAGAGTAGAAATGACTGC...","[(CGCAGAGTAGAAAT-AC-GCAG, CGCAGAGTAGAAATGACTGC...",CGCAGAGTAGAAAT-AC-GCAG\n|||||||||||||| || ||||...,CGCAGAGTAGAAAT-AC-GCAG\n|||||||||||||| || ||||...,i:15:G|i:17:T,i,2.0
5,GTTAGACGCAAAAAGATGAGACGCTGG,6.562247e-03,5.023884e-03,8.605874e-03,False,TGG,True,0,"[(CGCAGAGTAGAAATA-CGCAG, CGCAGAGTAGAAA-AACGCAG...","[(CGCAGAGTAGAAATACGCAG, CGCAGAGTAGAAAAACGCAG, ...",CGCAGAGTAGAAATACGCAG\n|||||||||||||.||||||\nCG...,CGCAGAGTAGAAATA-CGCAG\n||||||||||||| | |||||\n...,r:14:A,r,1.0
6,TTTAGATGCATAAAGATGAGGCGCTGG,5.423274e-04,4.393915e-04,8.173517e-04,False,TGG,True,0,"[(CGCAG-AGTAGAAATACGC-AG, CGC-GGAGTAGAAATACG-T...","[(CGCAGAGTAGAAATACGCAG, CGCGGAGTAGAAATACGTAG, ...",CGCAGAGTAGAAATACGCAG\n|||.|||||||||||||.||\nCG...,CGCAG-AGTAGAAATACGC-AG\n||| | |||||||||||| ||...,r:4:G|r:18:T,r,2.0
7,TTTAGACGCATAAAGATGAGACGCTGG,1.824571e-01,1.118804e-01,1.912647e-01,True,TGG,True,0,"[(CGCAGAGTAGAAATACGCAG, CGCAGAGTAGAAATACGCAG, ...","[(CGCAGAGTAGAAATACGCAG, CGCAGAGTAGAAATACGCAG, ...",CGCAGAGTAGAAATACGCAG\n||||||||||||||||||||\nCG...,CGCAGAGTAGAAATACGCAG\n||||||||||||||||||||\nCG...,OT,OT,0.0
8,TTATCTGCGTATTTCTACTCTGCGACC,8.303863e-28,2.254155e-29,1.910099e-20,False,ACC,False,0,"[(CGCAGAGTAGAAA-T-A-C-----GCAG--, -GC---GT----...","[(CGCAGAGTAGAAATACGCAG, GCGTCTCATCTTTATGCGTC, ...",CGCAGAGTAGAAATACGCAG\n....................\nGC...,CGCAGAGTAGAAA-T-A-C-----GCAG--\n || || ...,r:1:G|r:2:C|r:3:G|r:4:T|r:5:C|r:6:T|r:7:C|r:8:...,r,20.0
9,TTTAGACGCATAAACTACTCTGCCTGG,1.056168e-22,2.370000e-28,1.838904e-06,False,TGG,True,0,"[(CGCAGAGT---AG--AAATACGCAG, C-C---GTCTCA-TCAA...","[(CGCAGAGTAGAAATACGCAG, CCGTCTCATCAAATACGCAG, ...",CGCAGAGTAGAAATACGCAG\n|.........||||||||||\nCC...,CGCAGAGT---AG--AAATACGCAG\n| | || | ||||...,r:2:C|r:3:G|r:4:T|r:5:C|r:6:T|r:7:C|r:8:A|r:9:...,r,9.0
